In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

/home/janek/.local/share/virtualenvs/rosetta-WKmHhL03/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
import keras.layers as L
from keras.models import Model
import numpy as np
import pandas as pd

RANDOM_STATE=42
np.random.seed(RANDOM_STATE)
tf.set_random_seed(RANDOM_STATE)

In [3]:
START = '^'
END = '$'

SIZE = 100_000
LATENT_DIM = 512
EMBEDDING_DIM = 16
EPOCHS = 20
BATCH_SIZE = 128
DROPOUT = 0.5

In [4]:
def loguniform_int(low=0, high=1, size=1):
    offset = np.max([1 - low, 0])
    low, high = np.log([low + offset, high + offset])
    return (np.exp(np.random.uniform(low, high, size)) - offset).astype(int)

def create_equations_df(size, min_value=0, max_value=9999, operations={'+': np.add, '-': np.subtract}):
    df = pd.DataFrame()
    df['a'] = loguniform_int(low=min_value, high=max_value, size=size)
    df['b'] = loguniform_int(low=min_value, high=max_value, size=size)
    df['op'] = np.random.choice(list(operations.keys()), size)
    df['result'] = np.zeros(size, dtype='int')
    for symbol, calc in operations.items():
        df.loc[df.op == symbol, 'result'] = calc(df[df.op == symbol]['a'], df[df.op == symbol]['b'])
        
    df['input_texts'] = df.a.astype(str) + df.op + df.b.astype(str) + END
    df['target_texts'] = START + df.result.astype(str) + END
    return df

In [5]:
df = create_equations_df(SIZE)

In [6]:
corpus = pd.concat([df.input_texts, df.target_texts])

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=None, filters=None, char_level=True)
tokenizer.fit_on_texts(corpus)
df['input_sequences'] = tokenizer.texts_to_sequences(df.input_texts)
df['target_sequences'] = tokenizer.texts_to_sequences(df.target_texts)

In [8]:
X = keras.preprocessing.sequence.pad_sequences(df.input_sequences, padding='post')
y = keras.preprocessing.sequence.pad_sequences(df.target_sequences, padding='post')
y_t_output = keras.utils.to_categorical(y[:,1:], num_classes=len(tokenizer.word_index)+1)
x_t_input = y[:,:-1]

max_len_input = X.shape[1]
max_len_target = x_t_input.shape[1]
nr_tokens = y_t_output.shape[2]

In [9]:
tokenizer.word_index
nr_tokens
y_t_output.shape
len(tokenizer.word_index)

{'$': 1,
 '1': 2,
 '2': 3,
 '^': 4,
 '3': 5,
 '4': 6,
 '-': 7,
 '5': 8,
 '0': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13,
 '+': 14}

15

(100000, 6, 15)

14

In [10]:
encoder_gru = L.Bidirectional(
    L.GRU(LATENT_DIM // 2, dropout=DROPOUT, return_state=True, name='encoder_gru'),
    name='encoder_bidirectional'
)
decoder_gru = L.GRU(LATENT_DIM, dropout=DROPOUT, return_sequences=True, return_state=True, name='decoder_gru')
decoder_dense = L.Dense(nr_tokens, activation='softmax', name='decoder_outputs')

shared_embedding = L.Embedding(nr_tokens, EMBEDDING_DIM, mask_zero=True, name='shared_embedding')

encoder_inputs = L.Input(shape=(max_len_input, ), dtype='int32', name='encoder_inputs')
encoder_embeddings = shared_embedding(encoder_inputs)
_, encoder_state_1, encoder_state_2 = encoder_gru(encoder_embeddings)
encoder_states = L.concatenate([encoder_state_1, encoder_state_2])

decoder_inputs = L.Input(shape=(max_len_target, ), dtype='int32', name='decoder_inputs')
decoder_mask = L.Masking(mask_value=0)(decoder_inputs)
decoder_embeddings_inputs = shared_embedding(decoder_mask)
decoder_embeddings_outputs, _ = decoder_gru(decoder_embeddings_inputs, initial_state=encoder_states) 
decoder_outputs = decoder_dense(decoder_embeddings_outputs)


model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

inference_encoder_model = Model(encoder_inputs, encoder_states)
    
inference_decoder_state_inputs = L.Input(shape=(LATENT_DIM, ), dtype='float32', name='inference_decoder_state_inputs')
inference_decoder_embeddings_outputs, inference_decoder_states = decoder_gru(
    decoder_embeddings_inputs, initial_state=inference_decoder_state_inputs
)
inference_decoder_outputs = decoder_dense(inference_decoder_embeddings_outputs)

inference_decoder_model = Model(
    [decoder_inputs, inference_decoder_state_inputs], 
    [inference_decoder_outputs, inference_decoder_states]
)

In [11]:
model.summary()
inference_decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     (None, 6)            0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 6)            0           decoder_inputs[0][0]             
__________________________________________________________________________________________________
encoder_inputs (InputLayer)     (None, 10)           0                                            
__________________________________________________________________________________________________
shared_embedding (Embedding)    multiple             240         encoder_inputs[0][0]             
                                                                 masking_1[0][0]                  
__________

In [12]:
model.compile(optimizer=keras.optimizers.Adam(clipnorm=1.), loss='categorical_crossentropy')

In [13]:
model.fit([X, x_t_input], y_t_output, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE)

Train on 90000 samples, validate on 10000 samples
Epoch 1/20
90000/90000 [==============================] - 19s 208us/step - loss: 1.4554 - val_loss: 1.2199
Epoch 2/20
90000/90000 [==============================] - 17s 189us/step - loss: 0.9731 - val_loss: 0.7166
Epoch 3/20
90000/90000 [==============================] - 16s 183us/step - loss: 0.5876 - val_loss: 0.4270
Epoch 4/20
90000/90000 [==============================] - 17s 183us/step - loss: 0.3584 - val_loss: 0.2481
Epoch 5/20
90000/90000 [==============================] - 16s 183us/step - loss: 0.2270 - val_loss: 0.1729
Epoch 6/20
90000/90000 [==============================] - 17s 184us/step - loss: 0.1653 - val_loss: 0.1450
Epoch 7/20
90000/90000 [==============================] - 17s 184us/step - loss: 0.1348 - val_loss: 0.1009
Epoch 8/20
90000/90000 [==============================] - 17s 184us/step - loss: 0.1136 - val_loss: 0.0855
Epoch 9/20
90000/90000 [==============================] - 17s 185us/step - loss: 0.0992 - val_

In [14]:
def decode_sequence(input_seq):
    states_value = inference_encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, max_len_target))
    target_seq[0, 0] = tokenizer.word_index[START]
    
    tokens = {idx: token for (token, idx) in tokenizer.word_index.items()}
    
    decoded_sequence = ''
    for i in range(max_len_target):
        output_tokens, output_states = inference_decoder_model.predict(
            [target_seq, states_value]
        )
        
        sampled_token_idx = np.argmax(output_tokens[0, 0, :])
        sampled_token = tokens.get(sampled_token_idx, '.')
        if sampled_token == END:
            break
        decoded_sequence += sampled_token
            
        target_seq[0, 0] = sampled_token_idx
        states_value = output_states
    
    return decoded_sequence 

In [15]:
for calc in [eq + '$' for eq in ['1+1', '9+11', '21+34', '359+468', '1359+468', '1-1', '19-1', '34-359', '1359-468']]:
    input_seq = keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([calc]), 
        padding='post', 
        maxlen=X.shape[1]
    )
    print(f"{calc}=got: {decode_sequence(input_seq)}, exp: {eval(calc[:-1])}")

1+1$=got: 2, exp: 2
9+11$=got: 20, exp: 20
21+34$=got: 55, exp: 55
359+468$=got: 827, exp: 827
1359+468$=got: 1827, exp: 1827
1-1$=got: 0, exp: 0
19-1$=got: 18, exp: 18
34-359$=got: -325, exp: -325
1359-468$=got: 891, exp: 891


In [16]:
for calc in df.input_texts[:10].tolist():
    input_seq = keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([calc]), 
        padding='post', 
        maxlen=X.shape[1]
    )
    print(f"{calc}=got: {decode_sequence(input_seq)}, exp: {eval(calc[:-1])}")

30-209$=got: -179, exp: -179
6350+127$=got: 6477, exp: 6477
846-24$=got: 822, exp: 822
247-92$=got: 155, exp: 155
3+27$=got: 30, exp: 30
3-427$=got: -424, exp: -424
0-2187$=got: -2187, exp: -2187
2914-1403$=got: 1511, exp: 1511
252+22$=got: 274, exp: 274
678-108$=got: 560, exp: 570
